# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.233695579595                   -0.50    8.0         
  2   -7.250205528706       -1.78       -1.40    1.0   7.76ms
  3   -7.251293452359       -2.96       -2.15    2.0   9.47ms
  4   -7.251319476169       -4.58       -2.43    2.0   9.61ms
  5   -7.251333610035       -4.85       -2.78    1.0   8.13ms
  6   -7.251337855187       -5.37       -3.18    1.0   7.56ms
  7   -7.251338728435       -6.06       -3.64    2.0   9.55ms
  8   -7.251338795470       -7.17       -4.21    1.0   8.25ms
  9   -7.251338798265       -8.55       -4.79    2.0   9.59ms
 10   -7.251338798666       -9.40       -5.30    2.0   9.48ms
 11   -7.251338798703      -10.44       -5.90    2.0   9.58ms
 12   -7.251338798704      -11.85       -6.27    2.0   64.9ms
 13   -7.251338798705      -12.36       -6.86    2.0   11.2ms
 14   -7.251338798705      -13.45       -7.21    3.0   11.5ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.03585441739993785
[ Info: Arnoldi iteration step 2: normres = 0.4880846681006486
[ Info: Arnoldi iteration step 3: normres = 0.46623907903094164
[ Info: Arnoldi iteration step 4: normres = 0.8511630501637809
[ Info: Arnoldi iteration step 5: normres = 0.43058429536849896
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (1.89e-01, 7.56e-02, 2.95e-01, 2.37e-01, 2.20e-02)
[ Info: Arnoldi iteration step 6: normres = 0.38807553378981746
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (6.53e-02, 1.76e-01, 3.12e-01, 1.01e-01, 7.49e-02)
[ Info: Arnoldi iteration step 7: normres = 0.06837296519764237
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (2.29e-03, 1.63e-02, 1.31e-02, 2.85e-02, 5.08e-02)
[ Info: Arnoldi iteration step 8: normres = 0.12621500570226757
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (1.26